
# `src/visualize/plot_statistical_bands.py` – Statistical Aggregator

## 1. Algorithmic Challenge: Aligning Heterogeneous Data
The raw dataset consists of $\sim 20,000$ distinct EoS curves. Each curve is sampled at different pressure steps and covers a different mass range.
*   **Problem**: We cannot simply average the dataframes column-wise. Row 50 in Curve A might be at $1.2 M_{\odot}$, while Row 50 in Curve B is at $1.9 M_{\odot}$.
*   **Solution**: **Grid-Based Interpolation**. The script defines common x-axis grids and maps every individual physical curve onto these grids using linear or spline interpolation.

## 2. The Aggregation Pipeline

### Step A: Grid Definition
The script defines universal evaluation vectors based on plotting limits in `const.py`:
1.  **Energy Grid**: `eps_common` (Log-spaced, $10^2 \to 10^4$ MeV/fm³). Used for the EoS panel.
2.  **Mass Grid**: `mass_common` (Linearly spaced, $0.1 \to 3.0 M_{\odot}$). Used for the Tidal Deformability panel.

### Step B: The Interpolation Loop
The script iterates through every unique `Curve_ID` in the dataset (using `tqdm` for progress tracking). For each curve:
1.  **EoS Interpolation ($P$ vs $\varepsilon$)**:
    *   Since pressure spans orders of magnitude, interpolation is performed in **Log-Log space**.
    *   $f_{eos} = \text{interp1d}(\log \varepsilon, \log P)$.
    *   The interpolated pressure vector is stored in a matrix row.
2.  **Tidal Interpolation ($\Lambda$ vs $M$)**:
    *   $\Lambda$ drops exponentially with mass. Interpolation is performed in **Semi-Log space**.
    *   $f_{lam} = \text{interp1d}(M, \log_{10} \Lambda)$.
    *   The result is stored in the Lambda matrix.
3.  **M-R Collection**:
    *   Radius is not a single-valued function of Mass (the curve turns back at $M_{max}$). Therefore, interpolation is impossible/ill-defined.
    *   Instead, the script simply collects raw $(R, M)$ points into massive arrays for density estimation later.

### Step C: Statistical Reduction
Once the matrices are populated (Shape: $[N_{curves}, N_{grid}]$), the script computes column-wise statistics:
*   **5th Percentile**: The lower bound of the 90% Confidence Interval (CI).
*   **50th Percentile**: The Median behavior.
*   **95th Percentile**: The upper bound of the 90% CI.

**Smoothing**: To prevent the bands from looking jagged due to finite sampling noise, a **Gaussian Filter** (`scipy.ndimage.gaussian_filter1d`, $\sigma=2.0$) is applied to the percentile vectors before plotting.

---

## 3. Panel Analysis

### Panel A: Equation of State ($P(\varepsilon)$)
*   **Method**: Plots the smoothed percentiles on Log-Log axes.
*   **Visual**:
    *   **Hadronic (Green)**: The band is wide at high densities, reflecting the large uncertainty in nuclear physics (Soft vs. Stiff models).
    *   **Quark (Magenta)**: The band is narrower and typically follows a slope $\gamma \approx 1$ (Conformal limit $P \propto \varepsilon/3$), visually distinct from the steeper hadronic rise.
*   **Constraint**: A dotted line marks the **Causal Limit** ($P = \varepsilon$), the absolute physical ceiling.

### Panel B: Mass-Radius ($M(R)$)
*   **Method**: **Kernel Density Estimation (KDE)**. Since we cannot interpolate $R(M)$, we estimate the probability density of the scatter cloud.
    *   Uses `scipy.stats.gaussian_kde` on the accumulated $(R, M)$ points.
    *   Draws contours at 10% and 100% density levels.
*   **Constraint**: The **Buchdahl Limit** region is shaded gray.
    $$ R < \frac{9}{4} \frac{GM}{c^2} \approx 2.25 \times 1.4766 \times M $$
    This visualizes the General Relativistic prohibition on static fluid spheres existing in this corner of phase space.

### Panel C: Tidal Deformability ($\Lambda(M)$)
*   **Method**: Plots the smoothed percentiles on Semi-Log axes.
*   **Physics Result**: This is the most crucial panel for the thesis argument.
    *   At $1.4 M_{\odot}$, the Green band (Hadronic) sits significantly *higher* than the Magenta band (Quark).
    *   This statistically confirms that Hadronic stars are "fluffier" (more deformable) than self-bound Quark stars, justifying why Model A (which sees $\Lambda$) succeeds where Model Geo fails.

---

## 4. Visual Styling
The script enforces the publication standards defined in `style_config.py`:
*   **Hatching**: Quark bands use `hatch='////'` to ensure distinguishability in black-and-white prints.
*   **Alpha Blending**: Bands use `alpha=0.25` to allow overlapping regions to be seen (the "Intersection").
*   **Unified Legend**: A custom legend is created using `matplotlib.patches.Patch` to represent the statistical bands, rather than labeling every single line.

## 5. Output Files
*   **`plots/fig_statistical_bands.pdf`**: A clean, statistical summary of the physical dataset, free from the visual clutter of plotting 20,000 individual lines.